# Camera check

In [1]:
import cv2

camera0 = '/dev/video0' # 0번 카메라 열기 (기본 웹캠)
camera1 = '/dev/video2' # 2번 카메라 열기 (USB 웹캠)
camera2 = '/dev/camera_c270' # udev 세팅 카메라 열기
camera3 = "http://192.168.0.26:5000/video_feed"
use_camera = 0   # 1 또는 2로만 바꾸면 됨

if use_camera == 0:
    cap = cv2.VideoCapture(camera0)
elif use_camera == 1:
    cap = cv2.VideoCapture(camera1)
elif use_camera == 2:
    cap = cv2.VideoCapture(camera2, cv2.CAP_V4L2)
elif use_camera == 3:
    cap = cv2.VideoCapture(camera3, cv2.CAP_FFMPEG)
else:
    raise ValueError("use_camera는 1 또는 2여야 합니다")


if not cap.isOpened():
    print("❌ 카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 프레임을 읽을 수 없습니다.")
        break

    # 영상 출력
    cv2.imshow("Camera", frame)

    # 'q' 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()

QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/

# Webcam brightness, Exposure Setting

In [1]:
import cv2
import subprocess
import threading
import ipywidgets as widgets
from IPython.display import display

DEVICE = "/dev/video0"

# ---------- v4l2 control ----------
def set_ctrl(name, value):
    subprocess.run(
        ["v4l2-ctl", "-d", DEVICE, "-c", f"{name}={value}"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

# ---------- sliders ----------
exposure_slider = widgets.IntSlider(
    value=40,
    min=10,
    max=200,
    step=1,
    description="Exposure",
    continuous_update=True
)

brightness_slider = widgets.IntSlider(
    value=-20,
    min=-64,
    max=64,
    step=1,
    description="Brightness",
    continuous_update=True
)

def on_exposure_change(change):
    set_ctrl("exposure_time_absolute", change["new"])

def on_brightness_change(change):
    set_ctrl("brightness", change["new"])

exposure_slider.observe(on_exposure_change, names="value")
brightness_slider.observe(on_brightness_change, names="value")

display(exposure_slider, brightness_slider)

# ---------- camera thread ----------
cap = cv2.VideoCapture(DEVICE, cv2.CAP_V4L2)
if not cap.isOpened():
    raise RuntimeError("카메라 열기 실패")

def camera_loop():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow("Camera (ESC to exit)", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

threading.Thread(target=camera_loop, daemon=True).start()


IntSlider(value=40, description='Exposure', max=200, min=10)

IntSlider(value=-20, description='Brightness', max=64, min=-64)

QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/

In [ ]:
# Check Camera Setting
import cv2
import subprocess
import time

camera = "/dev/video2" 

# ---------- camera param apply ----------
subprocess.run([
    "v4l2-ctl", "-d", camera,
    "-c", "auto_exposure=1",
    "-c", "exposure_time_absolute=100",           # exposure
    "-c", "brightness=0",                         # brightness
    "-c", "gain=1",
    "-c", "backlight_compensation=0"
], check=True)

time.sleep(0.1)  # 드라이버 반영 시간 (중요)

# ---------- open camera ----------
cap = cv2.VideoCapture(camera, cv2.CAP_V4L2)

if not cap.isOpened():
    print("❌ 카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 프레임을 읽을 수 없습니다.")
        break

    cv2.imshow("Camera", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Flask Camera check

In [ ]:
from flask import Flask, Response
import cv2

camera0 = '/dev/video0' # 0번 카메라 열기 (기본 웹캠)
camera1 = '/dev/video2' # 2번 카메라 열기 (USB 웹캠)
camera2 = '/dev/camera_c270' # udev 세팅 카메라 열기

app = Flask(__name__)
cap = cv2.VideoCapture(camera0)  # USB camera

def generate():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        _, jpeg = cv2.imencode('.jpg', frame)
        frame_bytes = jpeg.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' +
               frame_bytes + b'\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(
        generate(),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000, threaded=True)

# flask_cam_test.py 에서 확인


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.26:5000
Press CTRL+C to quit
192.168.0.26 - - [05/Feb/2026 16:57:23] "GET /video_feed HTTP/1.1" 200 -


# 해상도 확인

In [1]:
import cv2

# 0번 카메라 열기 (필요 시 1, 2 등으로 변경)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
else:
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print(f"현재 카메라 해상도: {int(width)} x {int(height)}")

cap.release()


현재 카메라 해상도: 640 x 480


In [2]:
# Camera Frame Check
import cv2
import time


# 0번 카메라 열기 (기본 웹캠)
cap = cv2.VideoCapture('/dev/video0')

frame_count = 0
start_time = time.time()

if not cap.isOpened():
    print("❌ 카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 프레임을 읽을 수 없습니다.")
        break

    # while 루프 안 맨 아래에 추가
    frame_count += 1
    if frame_count % 30 == 0:  # 30프레임마다 한번 출력
        elapsed = time.time() - start_time
        fps = frame_count / elapsed
        print(f"📸 Current FPS: {fps:.2f}")


    # 영상 출력
    cv2.imshow("Camera", frame)

    # 'q' 키 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/genvenv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/choigh/venv/

📸 Current FPS: 30.99
📸 Current FPS: 30.48
📸 Current FPS: 30.32
📸 Current FPS: 30.24


ioctl(VIDIOC_QBUF): Bad file descriptor
